In [3]:
import numpy as np
import pandas as pd
from scipy.fft import fft, fftshift
import os

In [4]:
# Set load and save paths
load_filename = './data/raw/hybridsubset'
save_filename = './data/processed/'

In [5]:
# Parameters
BUI = {
    '00000': ['00000'],                             # Label 0: RF background activities
    '11000': ['11000'],                             # Label 1: Phantom drone RF activities
    '10100': ['10100', '10101', '10110', '10111'],  # Label 2: AR drone RF activities
    '10000': ['10000', '10001', '10010', '10011']   # Label 3: Bebop drone RF activities
}
M = 2048  # Total number of frequency bins
L = int(1e5)  # Total number of samples in a segment
Q = 10    # Number of returning points for spectral continuity

existing_files = os.listdir(load_filename)

combined_data = pd.DataFrame()


def process_data():
    global combined_data
    label_mapping = {'00000': 0, '11000': 1, '10100': 2, '10101': 2, '10110': 2, '10111': 2, 
                     '10000': 3, '10001': 3, '10010': 3, '10011': 3}

    for bui_key in BUI:
        for bui in BUI[bui_key]:
            print(f'Processing BUI: {bui}')
            if bui == '00000':
                N = 40  # Number of segments for RF background activities
            elif bui == '10111':
                N = 17
            else:
                N = 20  # Number of segments for drones RF activities

            data = []
            cnt = 0
            for n in range(N+1):
                x_file = f'{bui}L_{n}.csv'
                y_file = f'{bui}H_{n}.csv'

                # Check if the files exist in the directory
                if x_file not in existing_files or y_file not in existing_files:
                    print(f"Skipping missing files: {x_file} or {y_file}")
                    continue

                # Read the CSV files
                x = np.loadtxt(os.path.join(load_filename, x_file), delimiter=',')
                y = np.loadtxt(os.path.join(load_filename, y_file), delimiter=',')

                # Re-segment and signal transformation
                for i in range(len(x) // L):
                    st = i * L
                    fi = (i+1) * L

                    # FFT and signal processing
                    xf = np.abs(fftshift(fft(x[st:fi] - np.mean(x[st:fi]), M)))[M//2:]
                    yf = np.abs(fftshift(fft(y[st:fi] - np.mean(y[st:fi]), M)))[M//2:]

                    # Normalization and concatenation
                    processed_segment = np.concatenate((xf, yf * np.mean(xf[-Q:]) / np.mean(yf[:Q])), axis=0)
                    data.append(processed_segment)
                    cnt += 1

                print(f'Processed {100 * n / N:.2f}% of {bui}')

            # Save as .csv
            if len(data) > 0:
                processed_data = np.square(np.array(data))

                # Create labels
                labels = np.full((processed_data.shape[0], 1), label_mapping[bui])

                # Combine the data with the labels
                processed_data_with_labels = np.hstack((processed_data, labels))

                # Save to CSV
                output_file = os.path.join(save_filename, f'{bui}_processed.csv')
                pd.DataFrame(processed_data_with_labels).to_csv(output_file, index=False)
                print(f'Saved processed data to {output_file}')

                # Add this processed data to the combined DataFrame
                combined_data = pd.concat([combined_data, pd.DataFrame(processed_data_with_labels)], axis=0)
            else:
                print(f"No data processed for BUI: {bui}")

In [6]:
# Run the processing
process_data()

Processing BUI: 00000
Processed 0.00% of 00000
Processed 2.50% of 00000
Processed 5.00% of 00000
Processed 7.50% of 00000
Processed 10.00% of 00000
Processed 12.50% of 00000
Processed 15.00% of 00000
Processed 17.50% of 00000
Processed 20.00% of 00000
Processed 22.50% of 00000
Processed 25.00% of 00000
Processed 27.50% of 00000
Processed 30.00% of 00000
Processed 32.50% of 00000
Processed 35.00% of 00000
Processed 37.50% of 00000
Processed 40.00% of 00000
Processed 42.50% of 00000
Processed 45.00% of 00000
Processed 47.50% of 00000
Processed 50.00% of 00000
Processed 52.50% of 00000
Processed 55.00% of 00000
Processed 57.50% of 00000
Processed 60.00% of 00000
Processed 62.50% of 00000
Processed 65.00% of 00000
Processed 67.50% of 00000
Processed 70.00% of 00000
Processed 72.50% of 00000
Processed 75.00% of 00000
Processed 77.50% of 00000
Processed 80.00% of 00000
Processed 82.50% of 00000
Processed 85.00% of 00000
Processed 87.50% of 00000
Processed 90.00% of 00000
Processed 92.50% of 

In [7]:
# Save the combined data with labels as a single CSV file
combined_output_file = os.path.join(save_filename, 'combined_RF_Data_with_labels.csv')
combined_data.to_csv(combined_output_file, index=False)
print(f"Saved combined processed data with labels to {combined_output_file}")

Saved combined processed data with labels to ./data/processed/combined_RF_Data_with_labels.csv


In [8]:
combined_data.head(-5)

,0,1,2,3,4,5,6,7,8,9,...,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048
0,99960.383105,144491.277098,67509.687519,353678.589266,179610.092078,123225.905578,197539.154650,285239.244818,142365.877018,197117.080496,...,1004.351962,4693.246319,1538.236333,6141.352465,6526.354348,19096.299114,13558.563285,15270.059943,1440.036189,0.0
1,40085.037148,9588.252136,17058.554080,14719.488958,8072.615565,54905.210829,2213.120696,2269.927906,11432.638868,25970.552374,...,82253.106379,31905.369838,2020.913936,32602.369037,72551.388055,1359.821991,29112.643810,5565.234397,3409.809802,0.0
2,5934.520645,6878.466919,24978.202418,29597.225321,33215.379671,3628.769029,7709.433880,4127.557989,21248.001505,15278.984132,...,62628.103789,17192.164042,8687.768613,1253.662960,4238.062016,4287.809585,76401.390599,10133.728648,15652.232802,0.0
3,220.491613,30561.520557,24828.580215,28497.951697,530.545801,26647.271997,6501.346378,2641.342304,4269.118108,8496.036728,...,23280.337636,23196.938029,15740.890749,1489.863584,27643.843060,7607.918626,36330.522189,26000.615647,2133.041133,0.0
4,5568.454824,6520.865740,7974.086063,28381.931684,2955.276027,7602.680954,5148.909209,467.517616,2648.229659,38575.769022,...,3096.296217,14663.095154,3818.075756,9480.397571,16249.589306,11801.561673,5325.458477,10520.000611,80042.658260,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,8883.891919,1548.245414,5279.942188,24517.865494,117.723712,2023.078904,22829.836191,13008.322335,2884.490638,2667.588534,...,207.354756,10386.841465,5537.036238,18749.407250,1465.892795,8557.537705,9233.837539,24094.822686,26387.581224,2.0
291,28905.440256,33877.061509,6672.114108,7183.075012,1472.894554,3301.322982,42773.849728,23992.634268,1527.428514,5375.623190,...,8488.665336,103.818883,17091.975406,13446.634178,101035.750340,8132.870370,19997.785616,11159.278413,42125.536951,2.0
292,81107.576869,223732.421905,40296.108804,90583.814946,77232.737470,100205.201684,78253.903887,85492.818672,158482.453974,74795.942346,...,88889.065807,71211.116665,19113.559160,3101.422224,90932.948073,27780.177981,93.698794,388.780072,5244.012667,2.0
293,52477.793011,9727.095058,9713.654477,8294.331683,1199.695143,16432.746676,9359.644574,39582.364260,37052.571115,12292.400581,...,17485.331285,5609.918726,7551.499326,7998.454932,12900.855720,45981.292393,80656.794167,32521.998274,11254.233375,2.0


In [9]:
combined_data.shape

(4700, 2049)